In [90]:
import numpy as np
from typing import Tuple

In [91]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [101]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS)) # means the array is of 10 rows and 2 columns (bcz  10 num of items and 2 dimensions) with random integers between 0 and 100
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)) # means array is of 2 columns with random integer between 0 and 333

In [102]:
VALUES, WEIGHTS

(array([ 0, 43, 56,  5, 47, 70, 25, 85, 79, 70], dtype=int32),
 array([[44, 13],
        [91, 27],
        [23, 53],
        [13,  3],
        [11, 96],
        [ 6, 62],
        [42, 57],
        [96, 70],
        [26, 43],
        [70, 26]], dtype=int32))

In [103]:
print(f"Knapsack capacities (Constraints): \n{CONSTRAINTS}")

Knapsack capacities (Constraints): 
[[226 287]
 [249  45]
 [164 265]]


In [128]:
def solve_multidimensional_multiple_knapsack_greedy_optimized(
    weights_matrix: np.ndarray, 
    values: np.ndarray, 
    constraints_matrix: np.ndarray
) -> Tuple[float, np.ndarray]:
    """
    Optimized Greedy Heuristic for the Multidimensional Multiple Knapsack Problem (MMKP).
    
    Rule: Sort by Value/AverageWeight. Assign to the knapsack that maximizes 
          the total remaining capacity across all dimensions.
    """
    
    NUM_ITEMS = len(values)
    NUM_KNAPSACKS, NUM_DIMENSIONS = constraints_matrix.shape
    
    # 1. Prepare Data and Calculate Ratio
    item_weights = weights_matrix.astype(int)
    item_values = values.astype(float)
    
    # Use the average of all dimensions for the greedy ratio calculation.
    avg_weights = np.mean(item_weights, axis=1)
    
    ratios = np.divide(item_values, avg_weights, 
                       out=np.full_like(item_values, np.finfo(float).max, dtype=float), 
                       where=avg_weights != 0)
    
    # 2. Sort Items by Ratio (Descending Order)
    sorted_indices = np.argsort(ratios)[::-1]
    
    # 3. Initialize Tracking Variables
    current_capacities = constraints_matrix.copy().astype(float) 
    assignment_matrix = np.full((NUM_KNAPSACKS, NUM_ITEMS), False, dtype=bool)
    total_max_value = 0.0
    
    # 4. Greedy Assignment Loop
    for item_idx in sorted_indices:
        w_vec = item_weights[item_idx] # The weight vector for the current item
        v = item_values[item_idx]
        
        best_knapsack = -1
        max_remaining_capacity_sum = -1 # Used to find the best knapsack
        
        # Iterate over all knapsacks to find the best fit
        for j in range(NUM_KNAPSACKS):
            
            # 4a. Check Feasibility: Item must fit in ALL dimensions (The core MMKP check)
            if np.all(w_vec <= current_capacities[j]):
                
                # 4b. Greedy Tie-breaker Rule: Find the knapsack that leaves the most
                #     total remaining space across all dimensions after placing the item.
                
                remaining_capacity_vec = current_capacities[j] - w_vec
                
                # Sum the remaining capacity across all dimensions for this knapsack
                remaining_capacity_sum = np.sum(remaining_capacity_vec)
                
                if remaining_capacity_sum > max_remaining_capacity_sum:
                    max_remaining_capacity_sum = remaining_capacity_sum
                    best_knapsack = j
        
        # 5. Assign Item and Update State
        if best_knapsack != -1:
            # Assign the item to the best knapsack found
            assignment_matrix[best_knapsack, item_idx] = True
            
            # Subtract the weight vector from the capacity vector for that knapsack
            current_capacities[best_knapsack] -= w_vec
            total_max_value += v
            
    return total_max_value, assignment_matrix

print("--- Sequential Multiple Knapsack Solution with Assignments ---")
print(f"Item Values: {VALUES}")
print(f"Weights (Dim 0): {WEIGHTS}")
print(f"Capacities (Dim 0): {CONSTRAINTS}\n")

final_value, assignments = solve_multidimensional_multiple_knapsack_greedy_optimized(
    WEIGHTS, VALUES, CONSTRAINTS
)

print("\n--- Final Results ---")
print(f"Total Max Value Across All Knapsacks: {final_value:.2f}\n")

--- Sequential Multiple Knapsack Solution with Assignments ---
Item Values: [ 89 773 654 ... 819 678 498]
Weights (Dim 0): [[197  33 627 ... 319 937 293]
 [550 543 242 ... 942 568 950]
 [436 411 836 ... 664 169 846]
 ...
 [869 582 968 ... 567 291 288]
 [ 97 220 662 ... 171 310 833]
 [656  51 146 ... 286 725 372]]
Capacities (Dim 0): [[79834 52005 17527 ... 78500 56643 69915]
 [95067 28572 45458 ... 50816 83737 88901]
 [46612 36250 19604 ... 98589 28135 27193]
 ...
 [69672 90152 96973 ... 43906 41364 99258]
 [60364 83491 71527 ... 33636 13114 12777]
 [49805 58803 31629 ... 83824 73369 10897]]


--- Final Results ---
Total Max Value Across All Knapsacks: 1813475.00



In [129]:
print("Item Assignment Matrix (Rows = Knapsacks, Columns = Items):\n")
assignments

Item Assignment Matrix (Rows = Knapsacks, Columns = Items):



array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(100, 5000))

In [ ]:
# Check that the same object does not appear in multiple knapsacks
# this is done in the function itself 

In [131]:
# Check if the solution is valid

def validate_multiple_knapsack_solution(
    solution: np.ndarray, 
    weights_matrix: np.ndarray, 
    constraints_matrix: np.ndarray
) -> Tuple[bool, str]:
    # (The validation function is correct and doesn't need changes)
    NUM_KNAPSACKS, NUM_ITEMS = solution.shape
    
    # 1. Item Exclusivity Constraint (Assignment count per item <= 1)
    assignment_count_per_item = solution.sum(axis=0)
    if np.any(assignment_count_per_item > 1):
        overassigned_items = np.where(assignment_count_per_item > 1)[0]
        return False, f"Constraint Violated (Exclusivity): Items {overassigned_items} are assigned more than once."
    
    # 2. Knapsack Capacity Constraint (Load <= Capacity for all K, D)
    # Calculation: (K x I) @ (I x D) -> (K x D) load matrix
    current_load = solution @ weights_matrix
    
    capacity_check = current_load <= constraints_matrix
    
    if not np.all(capacity_check):
        violation_indices = np.where(capacity_check == False)
        knapsack_viol = violation_indices[0][0]
        dimension_viol = violation_indices[1][0]
        load = current_load[knapsack_viol, dimension_viol]
        cap = constraints_matrix[knapsack_viol, dimension_viol]
        
        return False, (
            f"Constraint Violated (Capacity): Knapsack {knapsack_viol} (Dim {dimension_viol}) "
            f"Load ({load}) exceeds Capacity ({cap})."
        )
    
    return True, "Solution is valid according to exclusivity and capacity constraints."


is_valid_1, msg_1 = validate_multiple_knapsack_solution(assignments, WEIGHTS, CONSTRAINTS)
print(f"\n[Valid Solution Check] Result: {is_valid_1}, Message: {msg_1}")



[Valid Solution Check] Result: True, Message: Solution is valid according to exclusivity and capacity constraints.


In [117]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [121]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [126]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))